Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

Begin Work - Feb 5, 2018 -mcliff

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [39]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [40]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.414817
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.366202
Minibatch accuracy: 43.8%
Validation accuracy: 55.2%
Minibatch loss at step 100: 0.800335
Minibatch accuracy: 75.0%
Validation accuracy: 72.7%
Minibatch loss at step 150: 0.552333
Minibatch accuracy: 87.5%
Validation accuracy: 76.4%
Minibatch loss at step 200: 0.970797
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%
Minibatch loss at step 250: 1.028398
Minibatch accuracy: 62.5%
Validation accuracy: 78.8%
Minibatch loss at step 300: 0.366895
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.405813
Minibatch accuracy: 93.8%
Validation accuracy: 77.3%
Minibatch loss at step 400: 0.288660
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%
Minibatch loss at step 450: 1.217271
Minibatch accuracy: 75.0%
Validation accuracy: 80.1%
Minibatch loss at step 500: 0.740700
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

Side Note: per this article we should never use sigmoids because they "saturate and die"
http://cs231n.github.io/neural-networks-1/

In [60]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data (X/y)
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

  #keep_prob = tf.placeholder(tf.float32) # dropout - keep probability

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #  5 x 5 x 1 x 16
  #   in the example I am looking at depth is 16
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    
  # 16 x 1
  layer1_biases = tf.Variable(tf.zeros([depth]))   

  # 5 x 5 x 16 x 16
  # in the example I have the 3rd and 4th numbers are 32 x 64
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))

  # 16 x 1
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  # ( (28//4)*(28//4)*(16) x 64 ) ->> 7*7*16 = 784 x 64
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  # 64 x 1
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  # 64 x 10
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  # 10 x 1
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    
  def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding="SAME")
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

  def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k,k,1], strides=[1,k,k,1], padding="SAME")

    
  # Model1
  def model1(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(mp1 + layer1_biases)
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    shape = hidden2.get_shape().as_list()
    reshape = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden3, layer4_weights) + layer4_biases
  
  # Model.
  def model(data):
    conv1 = conv2d(data, layer1_weights, layer1_biases)
    conv1 = maxpool2d(conv1, k=2)
    
    # apply dropout near start
    #conv1 = tf.nn.dropout(conv1, keep_prob)

    
    #conv1 = tf.nn.bias_add(conv1, layer1_biases)
    #hidden1 = tf.nn.relu(conv1)
    #hidden1 = tf.nn.max_pool(hidden1, ksize=[1,2,2,1], strides=[1,1,1,1], padding="SAME")
    #mp1 = tf.nn.max_pool(hidden1, ksize = [2,2,2,2], strides = [1,2,2,1], padding="SAME")
    #conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    #hidden2 = tf.nn.relu(conv2 + layer2_biases)
    #hidden2 = tf.nn.max_pool(hidden2, ksize=[1,2,2,1], strides=[1,1,1,1], padding="SAME")
    #shape = hidden2.get_shape().as_list()
    #reshape = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
    #hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #return tf.matmul(hidden3, layer4_weights) + layer4_biases
    conv2 = conv2d(conv1, layer2_weights, layer2_biases)
    conv2 = maxpool2d(conv2, k = 2)
    fc1 = tf.reshape(conv2, [-1, layer3_weights.get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, layer3_weights), layer3_biases)
    fc1 = tf.nn.relu(fc1)
    
    # apply dropout at the end
    #fc1 = tf.nn.dropout(fc1, keep_prob)
    return tf.add(tf.matmul(fc1, layer4_weights), layer4_biases)
    
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdamOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [61]:
num_steps = 1001
display_step = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    #feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.8}
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % display_step == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      #print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval({keep_prob: 1.0}), valid_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval({keep_prob: 1.0}), test_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.693205
Minibatch accuracy: 18.8%
Validation accuracy: 14.3%
Minibatch loss at step 100: 2.565991
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 200: 2.320209
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 300: 2.350258
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 400: 2.366362
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 500: 2.349413
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 600: 2.282361
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 700: 2.305832
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 800: 2.343302
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 900: 2.296824
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 2.275122
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Test 

In [53]:
def runSim(num_steps = 2001, print_step = 200):
  with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    for step in range(num_steps):
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.8}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % print_step == 0):
        print('Minibatch loss at step %d: %f' % (step, l))
        print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
        print('Validation accuracy: %.1f%%' % accuracy(
          valid_prediction.eval({keep_prob: 1.0}), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval({keep_prob: 1.0}), test_labels))
    
runSim()
runSim(num_steps=4001, print_step=400)

Minibatch loss at step 0: 2.813627
Minibatch accuracy: 6.2%
Validation accuracy: 9.8%
Minibatch loss at step 200: 1.035008
Minibatch accuracy: 75.0%
Validation accuracy: 75.6%
Minibatch loss at step 400: 0.383549
Minibatch accuracy: 81.2%
Validation accuracy: 77.3%
Minibatch loss at step 600: 0.527574
Minibatch accuracy: 81.2%
Validation accuracy: 75.4%
Minibatch loss at step 800: 0.894485
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 0.607804
Minibatch accuracy: 87.5%
Validation accuracy: 78.2%
Minibatch loss at step 1200: 1.946021
Minibatch accuracy: 43.8%
Validation accuracy: 76.8%
Minibatch loss at step 1400: 0.588909
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 1600: 1.256501
Minibatch accuracy: 56.2%
Validation accuracy: 80.4%
Minibatch loss at step 1800: 0.260595
Minibatch accuracy: 93.8%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.195956
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Test acc

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [38]:
num_steps = 2001
print_step = 200
dropout = 0.75  # keep 75%


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % print_step == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.779533
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 200: 0.963497
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 400: 0.226848
Minibatch accuracy: 100.0%
Validation accuracy: 81.3%
Minibatch loss at step 600: 0.387252
Minibatch accuracy: 81.2%
Validation accuracy: 82.6%
Minibatch loss at step 800: 0.511042
Minibatch accuracy: 81.2%
Validation accuracy: 84.7%
Minibatch loss at step 1000: 0.297534
Minibatch accuracy: 87.5%
Validation accuracy: 84.1%
Minibatch loss at step 1200: 0.886690
Minibatch accuracy: 62.5%
Validation accuracy: 85.0%
Minibatch loss at step 1400: 0.561622
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 1600: 0.926300
Minibatch accuracy: 81.2%
Validation accuracy: 85.8%
Minibatch loss at step 1800: 0.227944
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 2000: 0.106749
Minibatch accuracy: 93.8%
Validation accuracy: